In [38]:
#1.load the training data (numpy arrays of all the persons)
          # x - values are stored in numpy arrays
          # y - values we need to assign for each person
#2. Read Video using OpenCV
#3. extract face out of it
#4. use knn to find the prediction of face (int)
#5. map the predicted id to name of user
#6 . Display the prediction on the screen - bounding box and name

In [39]:
import cv2
import numpy as np
import os

In [40]:
from sklearn.neighbors import KNeighborsClassifier

face_pred=KNeighborsClassifier()


In [41]:
face_data = []
dataset_path ='./data/'
labels = []
class_id = 0 
names ={} # mapping between id-name

# Data Preaparation 

for fx in os.listdir(dataset_path):
    
    if fx.endswith('.npy'):
        print("Loaded : " + fx)
        
        # mapping between id-name
        names[class_id] = fx[:-4]
        
        
        data_item = np.load(dataset_path + fx)
        face_data.append(data_item)
    
        #Create Class Labels
        target = class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)
        
face_dataset = np.concatenate(face_data,axis = 0)
face_labels = np.concatenate(labels,axis = 0).reshape((-1,1))
print(face_dataset.shape)
print(face_labels.shape)


trainset = np.concatenate((face_dataset,face_labels),axis = 1)
print(trainset.shape)
face_pred.fit(trainset[:, :-1] , trainset[:,-1])

Loaded : abd.npy
Loaded : virat.npy
(610, 30000)
(610, 1)
(610, 30001)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [42]:
#init Camera
cap = cv2.VideoCapture('dataTest/abd and virat.mp4')

#Face Detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")



while True:
    
    ret,frame  = cap.read()
    
    if ret == False:
        continue
        
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    for face in faces:
        x,y,w,h = face
        
        #Extract (Crop out the required face) :  Region Of Interest
        offset = 10
        face_section = frame[y-offset:y+h+offset , x-offset:x+w+offset]
        face_section  = cv2.resize(face_section,(100,100))
        face_section = face_section.reshape((1,-1))
        
        #Predicted label out
        out=face_pred.predict(face_section)
        
        
        # Display on the screen the name and rectangle around it
        pred_name = names[int(out)]
        
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),2)
        
    cv2.imshow("Faces",frame)
    
     # Wait for user input - q ,then you will stop the loop 
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
        
        

        
        